## Generate training artifacts

In [12]:
from onnxruntime.training import artifacts
import torch
import onnx
import transformers
# from fastT5 import export_and_get_onnx_model
import onnxruntime.training.api as ort_api

In [2]:
# model = export_and_get_onnx_model("MBZUAI/LaMini-T5-61M")

In [3]:
# pipeline = transformers.pipeline(
#     "text-generation",
#     model="distilbert/distilgpt2",
# )

# transformers_model = transformers.AutoModelForCausalLM.from_pretrained("MBZUAI/LaMini-Cerebras-111M")
# transformers_model = transformers.AutoModel.from_pretrained("facebook/blenderbot-400M-distill")
transformers_model = transformers.AutoModel.from_pretrained("distilbert/distilgpt2")
# transformers_model_gpt = transformers.GPT2Model.from_pretrained("distilgpt2")

In [56]:
tokenizer = transformers.GPT2Tokenizer.from_pretrained("distilgpt2")
# tokenizer = transformers.GPT2Tokenizer.from_pretrained("distilbert/distilgpt2")
inputs = tokenizer("The capital of France is Paris.", return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 464, 3139,  286, 4881,  318, 6342,   13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}


In [5]:
class FlatModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, *local_inputs):
        return self.model(inputs.input_ids, inputs.attention_mask)

# model = FlatModel(pipeline.model)
model = FlatModel(transformers_model)

In [9]:
input_names = ["input_ids", "attention_mask"]
output_names = ["loss", "logits"]

torch.onnx.export(model,
                  (inputs["input_ids"], inputs["attention_mask"]),
                  "distilgpt2.onnx",
                  input_names = input_names, 
                  output_names = output_names,
                  export_params=True,
                  training=torch.onnx.TrainingMode.TRAINING,
                  do_constant_folding=False,
                  opset_version=15,
                  dynamic_axes={
                    "input_ids": {0: "batch_size", 1: "sequence_length"},
                    "attention_mask": {0: "batch_size", 1: "sequence_length"},
                    "logits": {0: "batch_size", 1: "sequence_length"}
                  }
                  )

input_ids: tensor([[ 464, 3139,  286, 4881,  318, 6342,   13]])
past key values: None
attention_mask tensor([[1, 1, 1, 1, 1, 1, 1]])


c:\Users\carolinezhu\AppData\Local\anaconda3\envs\generate-artifacts-new\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:804: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if batch_size <= 0:


In [11]:
requires_grad = []
frozen_params = []
num_named_params = 0
for name, param in model.named_parameters():
    num_named_params += 1
    if param.requires_grad:
        requires_grad.append(name)
    else:
        frozen_params.append(name)

for name, param in model.named_buffers():
    frozen_params.append(name)

# onnx_model = onnx.load("tinyllama.onnx")
onnx_model = onnx.load("distilgpt2.onnx")

artifacts.generate_artifacts(
    onnx_model,
    requires_grad=requires_grad,
    frozen_params=frozen_params,
    optimizer=artifacts.OptimType.AdamW,
    # loss=artifacts.LossType.CrossEntropyLoss,
)



args ('loss', 'logits', 'value.3', 'key.11', 'value.11', 'key.19', 'value.19', 'key.27', 'value.27', 'key.35', 'value.35', 'key.43', 'value.43')
args after edit ['loss', 'logits']


InferenceError: [ShapeInferenceError] (op_type:SoftmaxCrossEntropyLoss, node name: onnx::SoftmaxCrossEntropyLoss::10): labels typestr: Tind, has unsupported type: tensor(float)

## Data pre-processing

In [40]:
import numpy as np
import pandas as pd
from datasets import Dataset

In [15]:
df = pd.read_csv('../data/dialogueText.csv')

In [18]:
print(df.head)

<bound method NDFrame.head of          folder  dialogueID                      date         from  \
0             3  126125.tsv  2008-04-23T14:55:00.000Z    bad_image   
1             3  126125.tsv  2008-04-23T14:56:00.000Z    bad_image   
2             3  126125.tsv  2008-04-23T14:57:00.000Z    lordleemo   
3             3   64545.tsv  2009-08-01T06:22:00.000Z     mechtech   
4             3   64545.tsv  2009-08-01T06:22:00.000Z     mechtech   
...         ...         ...                       ...          ...   
1038319       3   51506.tsv  2012-01-31T10:56:00.000Z           DJ   
1038320       3   51506.tsv  2012-01-31T10:56:00.000Z     aeon-ltd   
1038321       3   99669.tsv  2008-11-16T20:11:00.000Z      KR-data   
1038322       3   99669.tsv  2008-11-16T20:12:00.000Z  outbackwifi   
1038323       3   99669.tsv  2008-11-16T20:13:00.000Z      KR-data   

                  to                                               text  
0                NaN  Hello folks, please help me a bit

In [42]:
current_convo =[]
convos = []
current_user = ""
current_responder = ""

for ind in df.index:
    if len(current_user) == 0 or str(df['from'][ind]) == current_user:
        # first convo OR continuing current user
        current_user = str(df['from'][ind])
        current_convo.append(
            {
                "role": "user", 
                "content": str(df['text'][ind])
            }
        )
    elif str(df['from'][ind]) != current_user and str(df['from'][ind]) != current_responder and len(current_responder) != 0:
        # new user & new convo
        convos.append(current_convo)
        current_convo = []
        current_user = str(df['from'][ind])
        current_responder = ""
        current_convo.append(
            {
                "role": "user", 
                "content": str(df['text'][ind])
            }
        )
    elif str(df['from'][ind]) != current_user:
        if str(df['from'][ind]) != current_responder:
            current_responder = str(df['from'][ind])
        current_convo.append(
            {
                "role": "assistant", 
                "content": str(df['text'][ind])
            }
        )
    else:
        print("ERROR: fall through -- you missed a case!", df['from'][ind], current_user, current_responder)

In [33]:
print(len(convos))
print(convos[3])

345692
[{'role': 'user', 'content': "is there any reason why my 'network manager' icon in my tray does not scale when the other ones get resized when i change the size of the panel?"}, {'role': 'assistant', 'content': 'probably not a vector icon'}, {'role': 'user', 'content': "the other icons aren't vector icons either."}]


In [54]:
llama_tokenizer = transformers.AutoTokenizer.from_pretrained("Xenova/TinyLlama-1.1B-Chat-v1.0")

In [55]:
dataset_dict = Dataset.from_dict({"chat": convos})
templated_convos_2 = dataset_dict.map(lambda x: {"formatted_chat": llama_tokenizer.apply_chat_template(x["chat"], tokenize=False, add_generation_prompt=False)})

Map: 100%|██████████| 345692/345692 [00:36<00:00, 9429.09 examples/s]


In [45]:
templated_convos[0]

{'chat': [{'content': "Hello folks, please help me a bit with the following sentence: 'Order here your personal photos or videos.' - I think the only allowed version is 'Order your personal videos or photos here.', but I'm not sure, are you?",
   'role': 'user'},
  {'content': 'Did I choose a bad channel? I ask because you seem to be dumb like windows user',
   'role': 'user'},
  {'content': 'the second sentence is better english   and we are not dumb',
   'role': 'assistant'}],
 'formatted_chat': "Hello folks, please help me a bit with the following sentence: 'Order here your personal photos or videos.' - I think the only allowed version is 'Order your personal videos or photos here.', but I'm not sure, are you?<|endoftext|>Did I choose a bad channel? I ask because you seem to be dumb like windows user<|endoftext|>the second sentence is better english   and we are not dumb<|endoftext|>"}

In [51]:
templated_convos_2[0]

{'chat': [{'content': "Hello folks, please help me a bit with the following sentence: 'Order here your personal photos or videos.' - I think the only allowed version is 'Order your personal videos or photos here.', but I'm not sure, are you?",
   'role': 'user'},
  {'content': 'Did I choose a bad channel? I ask because you seem to be dumb like windows user',
   'role': 'user'},
  {'content': 'the second sentence is better english   and we are not dumb',
   'role': 'assistant'}],
 'formatted_chat': "<|user|>\nHello folks, please help me a bit with the following sentence: 'Order here your personal photos or videos.' - I think the only allowed version is 'Order your personal videos or photos here.', but I'm not sure, are you?</s>\n<|user|>\nDid I choose a bad channel? I ask because you seem to be dumb like windows user</s>\n<|assistant|>\nthe second sentence is better english   and we are not dumb</s>\n"}

In [53]:
data_file = open("formatted_chat_500_convos.txt", "w")

data_file.writelines(templated_convos_2["formatted_chat"])

data_file.close()

## Test training artifacts by using the Python training API

In [13]:
## test generated artifacts
# create checkpoint state

state = ort_api.CheckpointState.load_checkpoint("checkpoint")

model = ort_api.Module('training_model.onnx', state, 'eval_model.onnx')

optimizer = ort_api.Optimizer('optimizer_model.onnx', model)

In [ ]:
tokenizer

In [60]:
tokenized_dataset = templated_convos_2.map(lambda x: tokenizer(x["formatted_chat"]))

Map:  57%|█████▋    | 195501/345692 [01:29<01:04, 2330.82 examples/s]

In [57]:
dataloader = torch.utils.data.DataLoader(tokenized_dataset, batch_size=64)

NameError: name 'tokenized_dataset' is not defined

In [ ]:
def train(epoch):
    model.train()
    losses = []
    for batch in dataloader:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        logits, loss = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        optimizer.step()
        model.lazy_reset_grad()
        losses.append(loss.item())
        